<a href="https://colab.research.google.com/github/Jake-Baum/model-gen/blob/main/PolyGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [3]:
import tensorflow as tf
import numpy as np
import urllib.request
import zipfile
import os

Load and unzip files

In [4]:
def unzip(readPath, writePath):
    with zipfile.ZipFile(readPath, 'r') as zip_ref:
        zip_ref.extractall(writePath)

url = 'https://masonmcgough-data-bucket.s3-us-west-2.amazonaws.com/ShapeNetCore_PolyGenSubset.zip'

urllib.request.urlretrieve(url, 'dataset.zip')
unzip('dataset.zip', 'dataset')

os.listdir('dataset')

['train', 'val']

Create method for loading objects from file into array of vertices.
Then a chair model is loaded

In [6]:
def loadObject(filename):
  vertices = []
  with open(filename, 'r') as mesh:
    for line in mesh:
      data = line.split()
      if len(data) > 0 and data[0] == 'v':
        vertices.append(data[1:])
  return np.array(vertices, dtype=np.float32)

verts = loadObject('dataset/train/chair/model_012429-0001.obj')
vertsKeys = [verts[..., i] for i in range(verts.shape[-1])]
sortIdxs = np.lexsort(vertsKeys)
vertsSorted = verts[sortIdxs]

Normalise (-1.0 - 1.0) and quantise (0, 255) vertices.

In [8]:
limits = [-1.0, 1.0]
normVerts = (verts - limits[0]) / (limits[1] - limits[0])

nVals = 2 ** 8
delta = 1. / nVals
quantVerts = np.maximum(np.minimum((normVerts // delta), nVals - 1), 0).astype(np.int32)

In [ ]:
TOKENS = {
    '<pad>': 0,
    '<sos>': 1,
    '<eos>': 2
}

maxVerts = 12
maxSeqLen = 3 * maxVerts + 2 # number of coords + start and stop tokens
nTokens = len(TOKENS)
seqLen = len(quantVerts) + 2
nPadding = maxSeqLen - seqLen

valTokens = np.concatenate((
    [TOKENS['<sos>']],
    quantVerts + nTokens,
    [TOKENS['<eos>']],
    nPadding * [TOKENS['<pad>']]
))

coordTokens = np.concatenate(([0], np.arrange(len(quantVerts)) % 3 + 1, (nPadding + 1) * [0]))
posTokens = np.arrange(len(quantTokens), dtype=np.int32)